## Loading data

In [6]:
import re
import pandas as pd

# Prepare data
path_pkl = os.path.join("data", "test_df.pkl")
data = pd.read_pickle(path_pkl) 

#data1 = data[:5000] 
#data2 = data[416537:]

#print(len(data1))
#print(len(data2))

#data = pd.concat([data1, data2])

print(len(data))

data = data[data["language"] == "en"]

print(len(data))

data = data[data["confidence"] > 0.9]

print(len(data))

# make text column lower case
data.document = data['document'].map(lambda document: document.lower() if isinstance(document,str) else document)

# make lists
dates = data.date.to_list()
docs = data.document.to_list()
subreddits = data.sub_reddit.to_list()
language = data.language.to_list()
type = data.type.to_list()

421537
408792
313281


In [4]:
data

,document,sub_reddit,date,type,language,confidence
1,"""why russians should take collective responsib...",ukraina,2022-03-23,submission,en,0.915159
2,reach out to russians and inform them about wa...,ukraina,2022-03-23,submission,en,0.916073
3,"march 23, 2022: ua air force destroyed 7 russi...",ukraina,2022-03-23,submission,en,0.810842
4,zelenskyy addresses in english: “i ask you to ...,ukraina,2022-03-23,submission,en,0.964464
5,europe will be next. truth about russian invas...,ukraina,2022-03-23,submission,en,0.901022
...,...,...,...,...,...,...
450184,it's on the home page articles today,geopolitics,2022-03-07,comment,en,0.896494
450185,it really is wild to see him so badly miscalcu...,geopolitics,2022-03-07,comment,en,0.997909
450186,panic fear and being cut off from swift \nrus...,geopolitics,2022-03-07,comment,en,0.973750
450187,post a [submission statement]( in one hour or ...,geopolitics,2022-03-07,comment,en,0.836011


### Checking if it works

In [405]:
pd.DataFrame.head(data)

data[:10]
data[9990:]

,document,sub_reddit,date,type,language,confidence
450179,thanks!,geopolitics,2022-03-07,comment,en,0.124504
450180,thanks!,geopolitics,2022-03-07,comment,en,0.124504
450181,"not sure if i was interpreting this wrong, but...",geopolitics,2022-03-07,comment,en,0.991418
450182,one thing i don’t understand is why ukraine is...,geopolitics,2022-03-07,comment,en,0.950550
450183,depends on the contract i assume. some might t...,geopolitics,2022-03-07,comment,en,0.954689
450184,it's on the home page articles today,geopolitics,2022-03-07,comment,en,0.896494
450185,it really is wild to see him so badly miscalcu...,geopolitics,2022-03-07,comment,en,0.997909
450186,panic fear and being cut off from swift \nrus...,geopolitics,2022-03-07,comment,en,0.973750
450187,post a [submission statement]( in one hour or ...,geopolitics,2022-03-07,comment,en,0.836011
450188,just because there are a few neo-nazis in a co...,geopolitics,2022-03-07,comment,en,0.974523


In [406]:
print(dates[-1])
print(docs[-1])

2022-03-07
just because there are a few neo-nazis in a country doesn't mean putin's justification for the invasion is true. the azov party got less than 2% of the votes in the 2019 election. zelensky is freaking jewish.

it's a bit of a disproportionate response to risk tens of thousands of lives and billions of dollars of equipment to remove a minor group.

if you don't see this as a completely transparent attempt to legitimize a power grab, then i don't think anything i can say will convince you.


In [407]:
#If this outputs anything, there is a problem. Something that should be a string is not...
for i in docs: 
    test = isinstance(i, str)
    if test == False: 
        print (test)

## Training embeddings

In [408]:
import random
random.seed(29)

#Husk at installe sentence_transformers via terminalen
from sentence_transformers import SentenceTransformer

# Prepare embeddings
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=False)


## Fitting the model

In [ ]:
#Kør kun hvis chunken nedenfor ikke virker
from umap import UMAP
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0, metric='cosine', random_state = 29)

In [409]:
#Skal virke for at få GPU'en til at køre hurtigere, men der er problemer med at loade cuml pakken

#from cuml.manifold import UMAP
#from cuml.cluster import HDBSCAN

# Create instances of GPU-accelerated UMAP and HDBSCAN
#umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0, metric='cosine', random_state = 29)
#hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True)

ModuleNotFoundError: No module named 'cuml'

In [ ]:


from bertopic import BERTopic

#Kør hvis GPU opsætning IKKE virker
topic_model = BERTopic(language = "multilingual", verbose = True, calculate_probabilities = False, umap_model=umap_model)
#Kør hvis GPU opsætning virker
#topic_model = BERTopic(language = "multilingual", verbose = True, calculate_probabilities = False, umap_model=umap_model, hdbscan_model=hdbscan_model)


topics, probs = topic_model.fit_transform(docs, embeddings)


2022-12-16 11:17:05,931 - BERTopic - Reduced dimensionality
2022-12-16 11:17:06,404 - BERTopic - Clustered reduced embeddings


### Removing stopwords 

In [ ]:
topic_model.get_topic_info()[0:11]

,Topic,Count,Name
0,-1,4295,-1_the_and_to_of
1,0,163,0_не_на_что_що
2,1,140,1_nuclear_nukes_nuke_use
3,2,104,2_nato_join_eu_membership
4,3,97,3_russians_are_they_people
5,4,93,4_him_he_sniff_little
6,5,82,5_he_him_his_amtrak
7,6,79,6_democracy_american_we_continuity
8,7,77,7_draft_military_you_your
9,8,72,8_drones_bayraktar_drone_tb2


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words="english")
#vectorizer_model = CountVectorizer(stop_words="english")

topic_model.update_topics(docs, vectorizer_model=vectorizer_model)

### Investigating the model

In [ ]:
topic_model.get_topic_info()[0:11]

,Topic,Count,Name
0,-1,4295,-1_russia_putin_russian_people
1,0,163,0_не_на_что_россии
2,1,140,1_nukes_nuclear_nuke_tactical
3,2,104,2_nato_join_eu_membership
4,3,97,3_russians_evil_people_change
5,4,93,4_sniff_little_kid_sniffing
6,5,82,5_amtrak_joe_elected_trump
7,6,79,6_democracy_american_continuity_autocracy
8,7,77,7_draft_military_year_platoon
9,8,72,8_drones_bayraktar_tb2_drone


### Saving and loading models

In [ ]:
#topic_model.save("models/model_medium")

In [ ]:
#from bertopic import BERTopic

#topic_model = BERTopic.load("models/model_medium")

## Visualizations

In [ ]:
#Tager lang tid at køre, når vi har så mange små topics
#topic_model.visualize_topics()

### Topics over time

### Function for visualizing topics over time

In [ ]:
import pandas as pd
from typing import List
import plotly.graph_objects as go
from sklearn.preprocessing import normalize


def visualize_topics_over_time(topic_model,
                               topics_over_time: pd.DataFrame,
                               top_n_topics: int = None,
                               topics: List[int] = None,
                               normalize_frequency: bool = False,
                               custom_labels: bool = False,
                               width: int = 1250,
                               height: int = 450) -> go.Figure:
    """ Visualize topics over time
    Arguments:
        topic_model: A fitted BERTopic instance.
        topics_over_time: The topics you would like to be visualized with the
                          corresponding topic representation
        top_n_topics: To visualize the most frequent topics instead of all
        topics: Select which topics you would like to be visualized
        normalize_frequency: Whether to normalize each topic's frequency individually
        custom_labels: Whether to use custom topic labels that were defined using 
                       `topic_model.set_topic_labels`.
        width: The width of the figure.
        height: The height of the figure.
    Returns:
        A plotly.graph_objects.Figure including all traces
    Examples:
    To visualize the topics over time, simply run:
    ```python
    topics_over_time = topic_model.topics_over_time(docs, timestamps)
    topic_model.visualize_topics_over_time(topics_over_time)
    ```
    Or if you want to save the resulting figure:
    ```python
    fig = topic_model.visualize_topics_over_time(topics_over_time)
    fig.write_html("path/to/file.html")
    ```
    <iframe src="../../getting_started/visualization/trump.html"
    style="width:1000px; height: 680px; border: 0px;""></iframe>
    """
    colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']

    # Select topics based on top_n and topics args
    freq_df = topic_model.get_topic_freq()
    freq_df = freq_df.loc[freq_df.Topic != -1, :]
    if topics is not None:
        selected_topics = list(topics)
    elif top_n_topics is not None:
        selected_topics = sorted(freq_df.Topic.to_list()[:top_n_topics])
    else:
        selected_topics = sorted(freq_df.Topic.to_list())

    # Prepare data
    if topic_model.custom_labels_ is not None and custom_labels:
        topic_names = {key: topic_model.custom_labels_[key + topic_model._outliers] for key, _ in topic_model.topic_labels_.items()}
    else:
        topic_names = {key: value[:40] + "..." if len(value) > 40 else value
                       for key, value in topic_model.topic_labels_.items()}
    topics_over_time["Name"] = topics_over_time.Topic.map(topic_names)
    data = topics_over_time.loc[topics_over_time.Topic.isin(selected_topics), :].sort_values(["Topic", "Timestamp"])

    # Add traces
    fig = go.Figure()
    for index, topic in enumerate(data.Topic.unique()):
        trace_data = data.loc[data.Topic == topic, :]
        topic_name = trace_data.Name.values[0]
        words = trace_data.Words.values
        if normalize_frequency:
            y = normalize(trace_data.Frequency.values.reshape(1, -1))[0]
        else:
            y = trace_data.Frequency
        fig.add_trace(go.Scatter(x=trace_data.Timestamp, y=y,
                                 mode='lines',
                                 marker_color=colors[index % 20],
                                 hoverinfo="text",
                                 name=topic_name,
                                 hovertext=[f'<b>Topic {topic}</b><br>Words: {word}' for word in words]))

    # Styling of the visualization
    fig.update_xaxes(showgrid=True)
    fig.update_yaxes(showgrid=True)
    fig.update_layout(
        yaxis_title="Normalized Frequency" if normalize_frequency else "Frequency",
        title={
            'text': "<b>Topics over Time",
            'y': .95,
            'x': 0.40,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
                size=22,
                color="Black")
        },
        template="simple_white",
        width=width,
        height=height,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Rockwell"
        ),
        legend=dict(
            title="<b>Global Topic Representation",
        )
    )
    return fig

In [ ]:
# Topics over time
topics_over_time = topic_model.topics_over_time(docs, dates, nr_bins=20)
#topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

visualize_topics_over_time(topic_model, topics_over_time, top_n_topics=20)




# Visualize topics over time with the updated colors
#visualize_topics_over_time(model, topics_over_time)

4it [00:00,  4.96it/s]


## Topics pr class

In [ ]:
# Topics per class
#topics_per_class = topic_model.topics_per_class(docs, classes = subreddits)

language = data.language.to_list()
type = data.type.to_list()

topics_per_class = topic_model.topics_per_class(docs, classes = language)




36it [00:00, 57.89it/s]


In [ ]:
topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10)
